In [ ]:
import requests as r
import bs4
import urllib
from newspaper import Article
import spacy
import re
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:

# !python -m spacy download en_core_web_sm


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
url_main = "https://www.google.com/search"
query = "RCB"
tbm = "nws"
start = "15"

URL = f"{url_main}?q={query}&tbm={tbm}&start={start}"


In [ ]:
url = "https://economictimes.indiatimes.com/small-biz/sme-sector/5-ways-your-company-can-be-carbon-neutral/articleshow/91431060.cms"


def get_html(url):
    response = r.get(url)
    return response.content


def get_content(html):
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup.find_all('p')


def get_text(content) -> list:
    text = []
    for i in content:
        text.append(i.text)
    return text


def check_url_validity(urls) -> bool:
    for i in urls:
        req = r.get(i)
        return True if req.status_code == 200 else False


def clean_urls(urls) -> list:
    cleaned_urls = []
    for i in urls:
        if i.startswith('/url?q=https://www'):
            link = i.replace('/url?q=', '')
            cleaned_urls.append(link)
    return cleaned_urls


def get_hrefs(html) -> list:
    soup = bs4.BeautifulSoup(html, 'html.parser')
    hrefs = []
    for i in soup.find_all('a'):
        hrefs.append(i.get('href'))
    return hrefs


def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text


def list_to_string(list) -> str:
    return ' '.join(list)


def remove_special_characters(text) -> str:
    cleaned_text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    return cleaned_text


def get_keywords(text, count=None) -> list:
    doc = nlp(text)
    words = []
    for token in doc:
        if token.is_stop != True:
            words.append(token.text)

    keywords = list(set(words))
    return len(keywords) if count else keywords

def get_domain_name(url) -> str:
    return urllib.parse.urlparse(url).netloc



In [ ]:

html = get_html(url)
content = get_content(html)
text = list_to_string(get_text(content))

cleaned_text = remove_special_characters(text)

keywords = get_keywords(cleaned_text)
keywords

In [106]:
def scrape_rss():
    url = "https://www.thehindu.com/news/national/tamil-nadu/feeder/default.rss"
    # url = "https://timesofindia.indiatimes.com/rssfeeds/296589292.cms"
    response = r.get(url)
    soup = bs4.BeautifulSoup(response.content, 'xml')
    news = soup.find_all('item')
    result = []
    for i in news:
        title = i.title.text
        category = i.category.text
        link = i.link.text
        description = i.description.text
        pub_date = i.pubDate.text
        # article_content = get_article(link)
        item = {
            'title': title.strip(),
            'category': category.strip(),
            'link': link.strip(),
            'description': description.strip(),
            'pubDate': pub_date.strip(),
            # 'article_content': article_content.strip()
        }
        result.append(item)
    return result

# scrape_rss()


In [107]:
rss_df = pd.DataFrame(scrape_rss())

In [108]:
rss_df

,title,category,link,description,pubDate
0,FDI flow into Tamil Nadu increases 30.1% in 20...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,,"Sun, 29 May 2022 23:57:26 +0530"
1,Diesel price cut has not brought much cheer to...,Chennai,https://www.thehindu.com/news/cities/chennai/d...,Freight charges don’t go up as much as they sh...,"Sun, 29 May 2022 23:54:15 +0530"
2,77 persons test positive for COVID-19 in State,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,"Chennai, Chengalpattu account for 62 cases; 41...","Sun, 29 May 2022 23:37:43 +0530"
3,Rumblings in T.N. BJP over Annamalai’s leadership,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,A section of leaders and some party functionar...,"Sun, 29 May 2022 23:36:29 +0530"
4,COVID-19 cluster at VIT Chennai campus,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,25 residential students test positive,"Sun, 29 May 2022 23:25:41 +0530"
5,Eight years of BJP rule has caused only misery...,New Articles,https://www.thehindu.com/incoming/eight-years-...,The party has not fulfilled any of its electio...,"Sun, 29 May 2022 22:49:55 +0530"
6,Chidambaram is Congress nominee for RS seat fr...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,,"Sun, 29 May 2022 22:20:51 +0530"
7,Handicrafts kiosk attracts PM’s attention,Tiruchirapalli,https://www.thehindu.com/news/cities/Tiruchira...,,"Sun, 29 May 2022 21:38:38 +0530"
8,Stanley Hospital offers master health checkup ...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,"New equipment, renovated operation theatres an...","Sun, 29 May 2022 21:15:50 +0530"
9,Children diagnosed with asthma can lead a norm...,Tamil Nadu,https://www.thehindu.com/news/national/tamil-n...,Doctors stress the need for right diagnosis an...,"Sun, 29 May 2022 20:56:29 +0530"
